In [1]:
"""
To make sure the fusion decomposition works
up to the part where we would stitch the sublimbs together into one limb


"""

'\nTo make sure the fusion decomposition works\nup to the part where we would stitch the sublimbs together into one limb\n\n\n'

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import skeleton_utils as sk
import soma_extraction_utils as sm
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import numpy as np
from importlib import reload
import networkx as nx
import time
import compartment_utils as cu
import networkx_utils as xu
import matplotlib_utils as mu
import neuron_utils as nru

#importing at the bottom so don't get any conflicts
import itertools
from tqdm_utils import tqdm

#for meshparty preprocessing
import meshparty_skeletonize as m_sk
import general_utils as gu
import compartment_utils as cu
from meshparty import trimesh_io
from copy import deepcopy


from neuron_utils import *


# Loading the Neuron

In [4]:
import trimesh_utils as tu
curent_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/Segmentation_2/864691135641957443_thick_dendrite.off")
segment_id = 864691136739250798
description = "double_soma_inhibit_stretched"

In [5]:
import neuron_visualizations as nviz
nviz.plot_objects(main_mesh=curent_neuron)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Setting the Arguments that would be present inside a preprocessing function call

In [6]:
#predefined arguments for the Neuron constructor

decomposition_type="meshafterparty"
mesh_correspondence="meshparty" #meshafterparty_adaptive
distance_by_mesh_center=True #how the distance is calculated for mesh correspondence
meshparty_segment_size = 100
meshparty_n_surface_downsampling = 2
meshparty_adaptive_correspondence_after_creation=False
suppress_preprocessing_print=True
computed_attribute_dict=None
somas = None
branch_skeleton_data=None
combine_close_skeleton_nodes = True
combine_close_skeleton_nodes_threshold=700
ignore_warnings=True
suppress_output=False
calculate_spines=True
widths_to_calculate=["no_spine_median_mesh_center",
                "no_spine_mean_mesh_center"]
fill_hole_size=2000

In [7]:
#arguments for the preprocess neuron
mesh = curent_neuron
segment_id=segment_id
description=description

sig_th_initial_split=15 #for significant splitting meshes in the intial mesh split
limb_threshold = 2000 #the mesh faces threshold for a mesh to be qualified as a limb (otherwise too small)
filter_end_node_length=4001 #used in cleaning the skeleton during skeletonizations
return_no_somas = False

decomposition_type=decomposition_type
mesh_correspondence=mesh_correspondence
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size =meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling
somas=somas
branch_skeleton_data=branch_skeleton_data
combine_close_skeleton_nodes = combine_close_skeleton_nodes
combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold

# Setting up the metadata

In [8]:
whole_processing_tiempo = time.time()


"""
Purpose: To process the mesh into a format that can be loaded into the neuron class
and used for higher order processing (how to visualize is included)

"""
if description is None:
    description = "no_description"
if segment_id is None:
    #pick a random segment id
    segment_id = np.random.randint(100000000)
    print(f"picking a random 7 digit segment id: {segment_id}")
    description += "_random_id"


if mesh is None:
    if current_mesh_file is None:
        raise Exception("No mesh or mesh_file file were given")
    else:
        current_neuron = trimesh.load_mesh(current_mesh_file)
else:
    current_neuron = mesh

# ************************ Phase A: Soma and Limb Identification ********************************

In [9]:
# --- 1) Doing the soma detection
if somas is None:
    soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)
else:
    soma_mesh_list,run_time,total_soma_list_sdf = somas

# geting the soma centers
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
    if return_no_somas:
        return_value= soma_mesh_list_centers
    raise Exception("Processing of No Somas is not yet implemented yet")
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 5553 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/neuron_864691136739250798.off -o /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/neuron_864691136739250798_decimated.off -s /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/decimation_meshlab_25141931.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(202613, 3), faces.shape=(402168, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(202613, 3), faces.shape=(402168, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/neuron_864691136739250798_decimated_largest_piece.off
xvfb-r

removed temporary input file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_133332.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_133332_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/poisson_54455.mls is being deleted....
Inside sphere validater: ratio_val = 31.10617507823371
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(828, 3), faces.shape=(1633, 3))>, curr_side_len_check = True, curr_volume_check = False
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 9249 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_772709.off -o /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_772709_poisson.off -s /notebooks/Platinum_Decomp_Fusion/current_running/Poiss

removed temporary input file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_853128.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_853128_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/poisson_535956.mls is being deleted....
Inside sphere validater: ratio_val = 268.9137928198519
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(564, 3), faces.shape=(1115, 3))>, curr_side_len_check = True, curr_volume_check = False
----- working on mesh after poisson #4: <trimesh.Trimesh(vertices.shape=(6607, 3), faces.shape=(13214, 3))>
xvfb-run -n 8878 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/neuron_864691136739250798_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Decomp_Fusion/current_running/8

done exporting decimated mesh: neuron_864691136739250798_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0004723072052001953
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2, path_and_filename = /notebooks/Platinum_Decomp_Fusion/current_running/temp/86469113673925079806_fixed 
1
Finished CGAL segmentation algorithm: 0.2886390686035156
2) Finished: Generating CGAL segmentation for neuron: 0.35330772399902344
3) Staring: Generating Graph Structure and Identifying Soma using soma size threshold  = 3000
my_list_keys = [0, 1, 2, 3, 4, 5]
soma_index = -1
3) Finished: Generating Graph Structure and Identifying Soma: 0.001857757568359375
Not finding the apical because soma_only option selected
6) Staring: Classifying Entire Neuron
Total Labels 

removed temporary input file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_172626.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_172626_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/poisson_628405.mls is being deleted....
Inside sphere validater: ratio_val = 208.35265042000557
--->This soma mesh was not added because it did not pass the sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(646, 3), faces.shape=(1282, 3))>, curr_side_len_check = True, curr_volume_check = False
----- working on mesh after poisson #9: <trimesh.Trimesh(vertices.shape=(5031, 3), faces.shape=(10058, 3))>
xvfb-run -n 5000 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/current_running/864691136739250798/neuron_864691136739250798_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Decomp_Fusion/current_running/

removed temporary input file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_333210.off
removed temporary output file: /notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/neuron_333210_poisson.off
mesh.is_watertight = True
/notebooks/Platinum_Decomp_Fusion/current_running/Poisson_temp/poisson_380548.mls is being deleted....
Inside sphere validater: ratio_val = 4.574122359084395
Soma List = [<trimesh.Trimesh(vertices.shape=(5146, 3), faces.shape=(10091, 3))>]
soma_mesh_list_centers = [array([1115135.09288768,  370487.08202487, 1005400.73870968])]


In [10]:
#--- 2) getting the soma submeshes that are connected to each soma and identifiying those that aren't (and eliminating any mesh pieces inside the soma)

main_mesh_total = current_neuron


#finding the mesh pieces that contain the soma
#splitting the current neuron into distinct pieces
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False)

print(f"# total split meshes = {len(split_meshes)}")


#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list,
                                        split_meshes)

# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    """
    *** want to save these pieces that are inside of the soma***
    """

    non_soma_touching_meshes,inside_pieces = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split,
                                    return_inside_pieces = True)                                                      


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]


#     print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
#     print(f"# of inside pieces = {len(inside_pieces)}")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")






# total split meshes = 25
There were 24 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


In [11]:
tu = reload(tu)
#--- 3)  Soma Extraction was great (but it wasn't the original soma faces), so now need to get the original soma faces and the original non-soma faces of original pieces

#     sk.graph_skeleton_and_mesh(other_meshes=[soma_meshes])

"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    current_time = time.time()
    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250)
    print(f"Total time for Subtract Soam = {time.time() - current_time}")
    current_time = time.time()

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)

    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    print(f"Total time for Original_mesh_faces_map for mesh_pieces without soma= {time.time() - current_time}")
    current_time = time.time()
    soma_meshes = current_mesh.submesh([soma_faces],append=True,repair=False)

    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True,repair=False)

    print(f"Total time for Original_mesh_faces_map for somas= {time.time() - current_time}")
    current_time = time.time()

    # 3) Find all significant mesh pieces
    sig_non_soma_pieces,insignificant_limbs = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=limb_threshold,
                                                     return_insignificant_pieces=True)

    print(f"Total time for sig_non_soma_pieces= {time.time() - current_time}")
    current_time = time.time()

    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces

    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False)
    #take the top largest ones depending how many were originally in the soma list
    seperate_soma_meshes = seperate_soma_meshes[:len(soma_mesh_list)]
    soma_face_components = soma_face_components[:len(soma_mesh_list)]

    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes

    print(f"Total time for split= {time.time() - current_time}")
    current_time = time.time()



    soma_to_piece_connectivity = dict()
    soma_to_piece_touching_vertices = dict()
    soma_to_piece_touching_vertices_idx = dict()
    limb_root_nodes = dict()
    
    m_vert_graph = tu.mesh_vertex_graph(current_mesh)
    
    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces

        soma_to_piece_touching_vertices[i] = dict()
        for piece_index,piece_idx in enumerate(connected_mesh_pieces):
            limb_root_nodes[piece_idx] = connected_mesh_pieces_vertices[piece_index][0]
            
            """ Old way of finding vertex connected components on a mesh without trimesh function
            #find the number of touching groups and save those 
            soma_touching_graph = m_vert_graph.subgraph(connected_mesh_pieces_vertices_idx[piece_index])
            soma_con_comp = [current_mesh.vertices[np.array(list(k)).astype("int")] for k in list(nx.connected_components(soma_touching_graph))]
            soma_to_piece_touching_vertices[i][piece_idx] = soma_con_comp
            """
            
            soma_to_piece_touching_vertices[i][piece_idx] = tu.split_vertex_list_into_connected_components(
                                                vertex_indices_list=connected_mesh_pieces_vertices_idx[piece_index],
                                                mesh=current_mesh, 
                                                vertex_graph=m_vert_graph, 
                                                return_coordinates=True
                                               )
            
            
            
            

#         border_debug = False
#         if border_debug:
#             print(f"soma_to_piece_connectivity = {soma_to_piece_connectivity}")
#             print(f"soma_to_piece_touching_vertices = {soma_to_piece_touching_vertices}")


    print(f"Total time for mesh_pieces_connectivity= {time.time() - current_time}")

    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

print(f"# of insignificant_limbs = {len(insignificant_limbs)} with trimesh : {insignificant_limbs}")



# Lets have an alert if there was more than one soma disconnected meshes
if len(soma_touching_mesh_data.keys()) > 1:
    raise Exception("More than 1 disconnected meshes that contain somas")

current_mesh_data = soma_touching_mesh_data
soma_containing_idx = 0

#doing inversion of the connectivity and touching vertices
piece_to_soma_touching_vertices = gu.flip_key_orders_for_dict(soma_to_piece_touching_vertices)


# ****Soma Touching mesh Data has the branches and the connectivity (So this is where you end up skipping if you don't have somas)***



----Working on soma-containing mesh piece 0----

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(332537, 3), faces.shape=(663687, 3))>, <trimesh.Trimesh(vertices.shape=(161155, 3), faces.shape=(321338, 3))>, <trimesh.Trimesh(vertices.shape=(77239, 3), faces.shape=(154083, 3))>, <trimesh.Trimesh(vertices.shape=(67220, 3), faces.shape=(134162, 3))>, <trimesh.Trimesh(vertices.shape=(57897, 3), faces.shape=(115303, 3))>, <trimesh.Trimesh(vertices.shape=(46947, 3), faces.shape=(93710, 3))>, <trimesh.Trimesh(vertices.shape=(30816, 3), faces.shape=(61437, 3))>, <trimesh.Trimesh(vertices.shape=(11382, 3), faces.shape=(22686, 3))>]
There were 8 pieces found after size threshold
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(332537, 3), faces.shape=(663687, 3))>, <trimesh.Trimesh(vertices.shape=(161155, 3), faces.shape=(321338, 3))>, <trimesh.Trimesh(vertices.shape=(772

# Process that will start for each limb

In [27]:
sk = reload(sk)
tu = reload(tu)
cu = reload(cu)
m_sk = reload(m_sk)
pre = reload(pre)
xu = reload(xu)
nu = reload(nu)
gu = reload(gu)


proper_time = time.time()

#The containers that will hold the final data for the preprocessed neuron
limb_correspondence=dict()
limb_network_stating_info = dict()

# ---------- Part A: skeletonization and mesh decomposition --------- #
skeleton_time = time.time()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    
    #Arguments to pass to the specific function (when working with a limb)
    soma_touching_vertices_dict = piece_to_soma_touching_vertices[curr_limb_idx]
    
#     if curr_limb_idx != 10:
#         continue
    
    curr_limb_time = time.time()
    print(f"\n\n----- Working on Proper Limb # {curr_limb_idx} ---------")
    

    limb_correspondence_individual,network_starting_info = pre.preprocess_limb(mesh=limb_mesh_mparty,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                   return_concept_network = False, 
                   return_concept_network_starting_info=True,
                   width_threshold_MAP=500,
                   size_threshold_MAP=2000,
                   surface_reconstruction_size=1000,                                                            
                   )
    #Storing all of the data to be sent to 
    
    limb_correspondence[curr_limb_idx] = limb_correspondence_individual
    limb_network_stating_info[curr_limb_idx] = network_starting_info
    
#     raise Exception("Done with #10")
    
    
print(f"Total time for Skeletonization and Mesh Correspondence = {time.time() - skeleton_time}")

pre=reload(pre)
neuron=reload(neuron)
nru = reload(nru)
# ---------- Part B: Stitching on floating pieces --------- #
floating_stitching_time = time.time()

limb_correspondence_with_floating_pieces = pre.attach_floating_pieces_to_limb_correspondence(
        limb_correspondence,
        floating_meshes=non_soma_touching_meshes,
        floating_piece_face_threshold = 600,
        max_stitch_distance=8000,
        distance_to_move_point_threshold = 4000,
        verbose = False)



print(f"Total time for stitching floating pieces = {time.time() - floating_stitching_time}")





# ---------- Part C: Computing Concept Networks --------- #
concept_network_time = time.time()

limb_concept_networks=dict()
limb_labels=dict()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    limb_to_soma_concept_networks = pre.calculate_limb_concept_networks(limb_correspondence_with_floating_pieces[curr_limb_idx],
                                                                        run_concept_network_checks=True,
                                                                       **limb_network_stating_info[curr_limb_idx])   



    limb_concept_networks[curr_limb_idx] = limb_to_soma_concept_networks
    limb_labels[curr_limb_idx]= "Unlabeled"
    
print(f"Total time for Concept Networks = {time.time() - concept_network_time}")





preprocessed_data= dict(
    soma_meshes = current_mesh_data[0]["soma_meshes"],
    soma_to_piece_connectivity = current_mesh_data[0]["soma_to_piece_connectivity"],
    soma_sdfs = total_soma_list_sdf,
    insignificant_limbs=insignificant_limbs,
    non_soma_touching_meshes=non_soma_touching_meshes,
    inside_pieces=inside_pieces,
    limb_correspondence=limb_correspondence_with_floating_pieces,
    limb_concept_networks=limb_concept_networks,
    limb_network_stating_info=limb_network_stating_info,
    limb_labels=limb_labels,
    limb_meshes=current_mesh_data[0]["branch_meshes"],
    )

    

print(f"Total time for all limb decomps = {time.time() - proper_time}")

#864049.29563888 1004924.982468    884750.28428994

neuron_obj = neuron.Neuron(
                mesh=curent_neuron,
                 segment_id=segment_id,
                 description=description,
                 preprocessed_data=preprocessed_data,
#                  decomposition_type="meshafterparty",
#                  mesh_correspondence="meshparty", #meshafterparty_adaptive
#                  distance_by_mesh_center=True, #how the distance is calculated for mesh correspondence
#                  meshparty_segment_size = 0,
#                  meshparty_n_surface_downsampling = 0,
#                  meshparty_adaptive_correspondence_after_creation=False,
#                 suppress_preprocessing_print=True,
#                  computed_attribute_dict=None,
#                  somas = None,
#                  branch_skeleton_data=None,
#                  combine_close_skeleton_nodes = True,
#                 combine_close_skeleton_nodes_threshold=700,
    
    
                ignore_warnings=True,
                suppress_output=False,
                calculate_spines=True,
                widths_to_calculate=["no_spine_median_mesh_center"]
                )



----- Working on Proper Limb # 0 ---------
Time for preparing soma vertices and root: 1.3589859008789062e-05
cc_vertex_thresh = 10


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:12<00:00, 12.19s/it]

Time for 1st pass MP skeletonization: 15.046976804733276
branches_touching_root = [106]


/meshAfterParty/meshparty_skeletonize.py:888: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  segment_branches = np.array([sk_meshparty_obj.vertices[np.vstack([k[:-1],k[1:]]).T] for k in segments])
/meshAfterParty/meshparty_skeletonize.py:921: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  new_segment_branches = np.array([sk_meshparty_obj.vertices[np.vstack([k[:-1],k[1:]]).T] for k in new_segments])
/meshAfterParty/meshparty_skeletonize.py:949: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-o

...

checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f30e48>]}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f30e48>]}
checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f305c0>]}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f305c0>]}
checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb96600f828>]}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb96600f828>]}
checking and resolving cycles
No cycles to fix
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f30c88>]}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7fb966f30c88>]}
checking and resolving cycles
No cycles to fix
--- 4) Finshed generating

Working on limb L5 branch 11
Working on limb L6 branch 1
Working on limb L6 branch 0
Working on limb L6 branch 2
Working on limb L6 branch 3
Working on limb L6 branch 4
Working on limb L7 branch 3
Working on limb L7 branch 2
Working on limb L7 branch 5
Working on limb L7 branch 1
Working on limb L7 branch 8
Working on limb L7 branch 4
Working on limb L7 branch 9
Working on limb L7 branch 0
Working on limb L7 branch 6
Working on limb L7 branch 7
Working on limb L7 branch 10
functions_list = [<function median_mesh_center at 0x7fba192320d0>, <function n_faces_branch at 0x7fba19230bf8>]



There was only one mesh found from the spine process and mesh split, returning empty array



Working on limb L0 branch 12
No spines and using precomputed width
Working on limb L0 branch 7
Working on limb L0 branch 20
No spines and using precomputed width
Working on limb L0 branch 30
Working on limb L0 branch 88
No spines and using precomputed width
Working on limb L0 branch 10
Working on limb L0 branch 51
Working on limb L0 branch 83
Working on limb L0 branch 84
Working on limb L0 branch 89
Working on limb L0 branch 90
No spines and using precomputed width
Working on limb L0 branch 11
Working on limb L0 branch 22
Working on limb L0 branch 50
No spines and using precomputed width
Working on limb L0 branch 54
No spines and using precomputed width
Working on limb L0 branch 41
Working on limb L0 branch 91
No spines and using precomputed width
Working on limb L0 branch 15
Working on limb L0 branch 16
Working on limb L0 branch 8
Working on limb L0 branch 71
Working on limb L0 branch 49
Working on limb L0 branch 56
Working on limb L0 branch 40
Working on limb L0 branch 75
No spines 

Working on limb L5 branch 15
Working on limb L5 branch 17
Working on limb L5 branch 4
Working on limb L5 branch 7
Working on limb L5 branch 14
Working on limb L5 branch 16
Working on limb L5 branch 0
No spines and using precomputed width
Working on limb L5 branch 12
Working on limb L5 branch 13
Working on limb L5 branch 6
Working on limb L5 branch 10
No spines and using precomputed width
Working on limb L5 branch 5
No spines and using precomputed width
Working on limb L5 branch 9
Working on limb L5 branch 8
No spines and using precomputed width
Working on limb L5 branch 11
No spines and using precomputed width
Working on limb L6 branch 1
Working on limb L6 branch 0
Working on limb L6 branch 2
Working on limb L6 branch 3
No spines and using precomputed width
Working on limb L6 branch 4
Working on limb L7 branch 3
No spines and using precomputed width
Working on limb L7 branch 2
No spines and using precomputed width
Working on limb L7 branch 5
No spines and using precomputed width
Workin

In [28]:
import neuron_visualizations as nviz

In [29]:
nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Sending the data to the Neuron Object

In [ ]:
nru = reload(nru)
save_time = time.time()
neuron_obj.save_compressed_neuron(output_folder="/notebooks/test_neurons/Fusion_decomp/",
                                 export_mesh=True,
                                 suppress_output=True)
print(f"Save time = {time.time() - save_time}")

In [ ]:
nru = reload(nru)
reload_time = time.time()
recovered_neuron = nru.decompress_neuron(filepath="./12345_double_soma",
                     original_mesh="./12345_double_soma")
print(f"Save time = {time.time() - reload_time}")

In [ ]:
import neuron_visualizations as nviz
returned_colors = nviz.visualize_neuron(recovered_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(L6="all"),
                                       return_color_dict=True)